In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading the data:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
submsn = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv')

In [ ]:
# Filling the missing values:
test['Age'].fillna((train['Age'].mean()), inplace=True)
train['Age'].fillna((train['Age'].mean()), inplace=True)
test['Fare'].fillna((train['Fare'].mean()), inplace=True)
train['Fare'].fillna((train['Fare'].mean()), inplace=True)

In [ ]:
# Dropping some columns:
train.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)
test.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)

In [ ]:
# Applying Label Encoder:
obj_cols = train.select_dtypes(include=['object']).columns.tolist()
for col in obj_cols:
    le = LabelEncoder()
    le.fit(train[col])
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [ ]:
# Defining the columns and target:
cols = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
target = train.Survived.values

# Fitting the model
model = HistGradientBoostingClassifier()
model.fit(train[cols], target)

In [ ]:
# Getting the score for train data:
roc_auc_score(target, model.predict_proba(train[cols])[:,1])

In [ ]:
# Getting the predictions for test data:
submsn['Survived'] = model.predict(test[cols])
submsn.to_csv('submission.csv', index=False)